In [3]:
from __future__ import print_function

import numpy as np
import SimpleITK as sitk
import six
import matplotlib.pyplot as plt
import os
import glob
from skimage import io

import csv
from radiomics import base, cShape, deprecated
from radiomics import firstorder, getTestCase, glcm, glrlm, glszm, shape2D,imageoperations, shape

import pandas as pd

In [4]:
#Lectura del directorio
directorio ="C:/Users/JUANK/Documents/GitHub/Estadistica-R/Proyecto_final/Dataset_Estadistica/segmentation_WBC"
directorio = os.path.join(directorio,'Dataset_2')
directorio


'C:/Users/JUANK/Documents/GitHub/Estadistica-R/Proyecto_final/Dataset_Estadistica/segmentation_WBC\\Dataset_2'

In [5]:
#aquí tengo los archivos del directorio
archivos = os.listdir(directorio)
archivos_comparar=list([os.path.splitext(x)[0] for x in archivos])
archivos_comparar.sort()

#archivos_comparar

In [6]:
filename = "C:/Users/prestamour/Downloads/segmentation_WBC"
filename = os.path.join(filename,"Class labels of Dataset 2" )

#necesito el 2 y el 3

indices_celulas=[]

with open('ClassLabelsOfDataset2.csv') as csvFile:
    reader = csv.reader(csvFile)
    for row in reader:
        if (row[1]== '2' or row[1]== '3'): 
            indices_celulas.append(int(row[0]))
        
csvFile.close()
#indices_celulas


In [7]:
# vamos a clasificar cuales son las celulas que nos sirven comparando archivos
#con nuestra nueva indices celulas

# archivos para trabajar
archivos_final = []
archivos_comparar=[int (x) for x in archivos_comparar ]

for i in indices_celulas:

    temp = archivos_comparar.index(i)
    archivos_final.append(archivos[temp])
    archivos_final.append(archivos[temp + 1])

#archivos_final

In [8]:
def cell_features (img,mask):
    result = pd.DataFrame(pd.Series({})).T 
    for j in range(3):
        #Escogiendo el color de la matriz
        img_color = img[:,:,j]
        img_final = sitk.GetImageFromArray(img_color)
        for k in range(len(nuc_cito)):
            #selección de nucleo o citoplasma
            n_c= mask == nuc_cito[k]
            n_c= 1*(n_c>0)
            mask_final = sitk.GetImageFromArray(n_c)

            #convertimos la imagen a 3D
            image3D=sitk.JoinSeries(img_final)         
            mask3D=sitk.JoinSeries(mask_final)
            
            #descriptores de primer orden 
            FOF= firstorder.RadiomicsFirstOrder(image3D,mask3D) #descriptores de primer orden por pyreadiomics
            FOF.enableAllFeatures()
            FOF.execute()
            fr= pd.DataFrame(pd.Series(FOF.featureValues)).T
            fr.columns = [s_nuc_cito[k]+'_' +x + '_'+ RGB[j] for x in fr.columns.to_list()]

            #descriptores de forma en 2D
            DSH = shape2D.RadiomicsShape2D(img_final, mask_final)
            DSH.enableAllFeatures()
            DSH.execute()
            fr2 = pd.DataFrame(pd.Series(DSH.featureValues)).T
            fr2.columns = [s_nuc_cito[k]+'_' +x + '_'+ RGB[j] for x in fr2.columns.to_list()]

            #concatenación
            result = pd.concat([result,fr],1)
            result = pd.concat([result,fr2],1)
            
            
            
    return result

In [9]:
#MAIN
#Definimos unas variables globales
indice_a= 0
RGB= ['R', 'G', 'B']
save = []
nuc_cito= [255,128]
s_nuc_cito=['nuc','cito']
resultados = 0
result_final = pd.DataFrame(pd.Series({})).T
#recorremos todas las imágenes
for i in archivos_final:
   
    if indice_a %2 == 0 and indice_a < len(archivos_final):
       
        img = io.imread(os.path.join(directorio, archivos_final[indice_a]) )
        mask = io.imread(os.path.join(directorio, archivos_final[indice_a + 1]))
        
        result = cell_features(img,mask)
        index = i[:-4]
        result.index =[index]
        result_final =  pd.concat([result_final,result])
    indice_a +=1

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [11]:
#list((result_final.columns).T)
#result_final